In [1]:
import pandas as pd
import numpy as np
import math
from scipy.spatial import cKDTree
from scipy.optimize import curve_fit
import warnings

warnings.filterwarnings("ignore")

In [2]:
main = pd.read_csv("data/total.csv")
water = pd.read_csv("data/water_velocity.csv")
wind = pd.read_csv("data/wind_velocity.csv")

In [3]:
water = water.fillna(0)
wind = wind.fillna(0)

In [4]:
main["water_u"] = 0
main["water_v"] = 0
main["wind_u"] = 0
main["wind_v"] = 0
main['d_long'] = main['longitude'].diff().fillna(0)
main['d_lati'] = main['latitude'].diff().fillna(0)

main = main.groupby('drifter').apply(lambda group: group.iloc[1:]).reset_index(drop=True)

In [5]:
# Create a grid of longitude and latitude from the water DataFrame
grid = water[['longitude', 'latitude']].drop_duplicates().values

# Create a KD-Tree for efficient nearest neighbor search
kdtree = cKDTree(grid)

# Function to find the 4 closest vertices for a given point
def find_closest_vertices(row):
    point = np.array([row['longitude'], row['latitude']])
    # Query the KD-Tree to find the 4 closest vertices
    _, indices = kdtree.query(point, k=4)
    return indices

# Apply the function to the main DataFrame to find the 4 closest vertices for each point
main['closest_vertices'] = main.apply(find_closest_vertices, axis=1)

In [6]:
main

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati,closest_vertices
0,0,51.50,129.401993,35.020000,0,0,0,0,0.090988,0.070000,"[1862, 1770, 1863, 1771]"
1,0,51.75,129.514008,35.061001,0,0,0,0,0.112015,0.041000,"[1863, 1862, 1771, 1770]"
2,0,52.00,129.524002,35.032001,0,0,0,0,0.009995,-0.028999,"[1863, 1862, 1771, 1770]"
3,0,52.25,129.626007,35.083000,0,0,0,0,0.102005,0.050999,"[1863, 1771, 1864, 1862]"
4,0,52.50,129.712997,35.058998,0,0,0,0,0.086990,-0.024002,"[1863, 1864, 1771, 1772]"
...,...,...,...,...,...,...,...,...,...,...,...
15809,30,638.00,147.348999,44.167999,0,0,0,0,0.029999,-0.000999,"[5243, 5335, 5151, 5427]"
15810,30,638.25,147.307007,44.138000,0,0,0,0,-0.041992,-0.029999,"[5243, 5335, 5151, 5427]"
15811,30,638.50,147.250000,44.146999,0,0,0,0,-0.057007,0.008999,"[5243, 5335, 5151, 5427]"
15812,30,638.75,147.322998,44.201000,0,0,0,0,0.072998,0.054001,"[5243, 5335, 5151, 5427]"


In [7]:
water

,time,longitude,latitude,u,v
0,0.00,123.875,30.125,0.086603,-0.309205
1,0.00,124.125,30.125,0.067756,-0.295678
2,0.00,124.375,30.125,0.081384,-0.253029
3,0.00,124.625,30.125,0.104257,-0.347986
4,0.00,124.875,30.125,0.097815,-0.409650
...,...,...,...,...,...
25520795,729.75,145.625,53.625,0.014556,-0.133729
25520796,729.75,145.875,53.625,0.006857,-0.136531
25520797,729.75,146.125,53.625,-0.000076,-0.133501
25520798,729.75,146.375,53.625,-0.004371,-0.127829


In [15]:
r = []
theta = []
for i in range(len(main)):
    x = water.loc[water["time"] == main.iloc[i][1]]
    w_d = []
    w_r = []
    w_theta = []
    
    for j in main.iloc[i][-1]:
        d = ((main.iloc[i][2] - x.iloc[j][1])**2 + (main.iloc[i][3] - x.iloc[j][2])**2) ** 0.5
        _r = ((x.iloc[j][-2] ** 2) + (x.iloc[j][-1] ** 2)) * 0.5
        
        u = np.array([x.iloc[j][-2], 0])
        v = np.array([0, x.iloc[j][-1]])
        dot_product = np.dot(u, u + v)
        u_norm = np.linalg.norm(u)
        u_plus_v_norm = np.linalg.norm(u+v)
        
        cos_theta = dot_product / (u_norm * u_plus_v_norm)
        _theta = math.acos(cos_theta)
        
        w_d.append(d)
        w_r.append(_r)
        w_theta.append(_theta)
        
    weights = [1 / d for d in w_d]
    total_weight = sum(weights)
    normalized_weights = [weight/total_weight for weight in weights]
    
    interpolated_r = sum(weight * r for weight, r in zip(normalized_weights, w_r))
    interpolated_theta = sum(weight * theta for weight, theta in zip(normalized_weights, w_theta))
    
    r.append(interpolated_r)
    theta.append(interpolated_theta)

In [16]:
w_u = []
w_v = []

for i in range(len(r)):
    u = r[i] * math.cos(theta[i])
    v = r[i] * math.sin(theta[i])
    w_u.append(u)
    w_v.append(v)
    
main["water_u"] = w_u
main["water_v"] = w_v

In [17]:
main

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati,closest_vertices
0,0,51.50,129.401993,35.020000,0.010100,0.008820,0,0,0.090988,0.070000,"[1862, 1770, 1863, 1771]"
1,0,51.75,129.514008,35.061001,0.008299,0.007565,0,0,0.112015,0.041000,"[1863, 1862, 1771, 1770]"
2,0,52.00,129.524002,35.032001,0.014182,0.013137,0,0,0.009995,-0.028999,"[1863, 1862, 1771, 1770]"
3,0,52.25,129.626007,35.083000,0.046504,0.029970,0,0,0.102005,0.050999,"[1863, 1771, 1864, 1862]"
4,0,52.50,129.712997,35.058998,0.077739,0.017259,0,0,0.086990,-0.024002,"[1863, 1864, 1771, 1772]"
...,...,...,...,...,...,...,...,...,...,...,...
15809,30,638.00,147.348999,44.167999,0.003455,0.001410,0,0,0.029999,-0.000999,"[5243, 5335, 5151, 5427]"
15810,30,638.25,147.307007,44.138000,0.003334,0.001393,0,0,-0.041992,-0.029999,"[5243, 5335, 5151, 5427]"
15811,30,638.50,147.250000,44.146999,0.003553,0.000779,0,0,-0.057007,0.008999,"[5243, 5335, 5151, 5427]"
15812,30,638.75,147.322998,44.201000,0.003457,0.000841,0,0,0.072998,0.054001,"[5243, 5335, 5151, 5427]"


In [18]:
main.describe()

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati
count,15814.000000,15814.000000,15814.000000,15814.000000,14038.000000,14038.000000,15814.0,15814.0,15814.000000,15814.000000
mean,15.866384,341.630644,134.255299,38.418786,0.016812,0.017204,0.0,0.0,0.014679,0.006684
std,8.613671,140.691731,3.622366,2.460060,0.023001,0.021718,0.0,0.0,0.053495,0.043642
min,0.000000,51.500000,125.299004,33.140999,0.000026,0.000017,0.0,0.0,-0.230011,-0.247002
25%,9.000000,240.250000,131.248001,36.632000,0.004008,0.003910,0.0,0.0,-0.016998,-0.016998
50%,16.000000,348.000000,133.824501,37.919998,0.009310,0.009472,0.0,0.0,0.010010,0.006001
75%,23.000000,436.250000,136.547752,39.633999,0.020841,0.021563,0.0,0.0,0.043991,0.031002
max,30.000000,666.000000,148.697998,46.507000,0.637869,0.248313,0.0,0.0,0.475006,0.247002


In [10]:
# Create a grid of longitude and latitude from the water DataFrame
grid2 = wind[['longitude', 'latitude']].drop_duplicates().values

# Create a KD-Tree for efficient nearest neighbor search
kdtree = cKDTree(grid2)

# Function to find the 4 closest vertices for a given point
def find_closest_vertices(row):
    point = np.array([row['longitude'], row['latitude']])
    # Query the KD-Tree to find the 4 closest vertices
    _, indices = kdtree.query(point, k=4)
    return indices

# Apply the function to the main DataFrame to find the 4 closest vertices for each point
main['closest_vertices2'] = main.apply(find_closest_vertices, axis=1)

In [11]:
r = []
theta = []
for i in range(len(main)):
    x = wind.loc[wind["time"] == main.iloc[i][1]]
    w_d = []
    w_r = []
    w_theta = []
    
    for j in main.iloc[i][-1]:
        d = ((main.iloc[i][2] - x.iloc[j][1])**2 + (main.iloc[i][3] - x.iloc[j][2])**2) ** 0.5
        _r = ((x.iloc[j][-2] ** 2) + (x.iloc[j][-1] ** 2)) * 0.5
        
        u = np.array([x.iloc[j][-2], 0])
        v = np.array([0, x.iloc[j][-1]])
        dot_product = np.dot(u, u + v)
        u_norm = np.linalg.norm(u)
        u_plus_v_norm = np.linalg.norm(u+v)
        
        cos_theta = dot_product / (u_norm * u_plus_v_norm)
        _theta = math.acos(cos_theta)
        
        w_d.append(d)
        w_r.append(_r)
        w_theta.append(_theta)
        
    weights = [1 / d for d in w_d]
    total_weight = sum(weights)
    normalized_weights = [weight/total_weight for weight in weights]
    
    interpolated_r = sum(weight * r for weight, r in zip(normalized_weights, w_r))
    interpolated_theta = sum(weight * theta for weight, theta in zip(normalized_weights, w_theta))
    
    r.append(interpolated_r)
    theta.append(interpolated_theta)

In [12]:
for i in range(10):
    print(r[i])

9.271803543228579
9.281162174438954
0.9491897289957052
55.1656548031403
120.36314845100297
76.37248873801985
33.84641877673009
2.800054375295388
3.912444683041535
25.320573415050912


In [13]:
w_u = []
w_v = []

for i in range(len(r)):
    u = r[i] * math.cos(theta[i])
    v = r[i] * math.sin(theta[i])
    w_u.append(u)
    w_v.append(v)
    
main["wind_u"] = w_u
main["wind_v"] = w_v

In [14]:
main

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati,closest_vertices,closest_vertices2
0,0,51.50,129.401993,35.020000,0.006120,0.005480,8.846708,2.775265,0.090988,0.070000,"[1862, 1770, 1863, 1771]","[6830, 6738, 6831, 6739]"
1,0,51.75,129.514008,35.061001,0.007229,0.007424,8.619745,3.440925,0.112015,0.041000,"[1863, 1862, 1771, 1770]","[6739, 6831, 6738, 6830]"
2,0,52.00,129.524002,35.032001,0.014755,0.013264,0.923540,0.219167,0.009995,-0.028999,"[1863, 1862, 1771, 1770]","[6831, 6739, 6830, 6738]"
3,0,52.25,129.626007,35.083000,0.049938,0.025867,29.629408,46.533296,0.102005,0.050999,"[1863, 1771, 1864, 1862]","[6739, 6831, 6740, 6738]"
4,0,52.50,129.712997,35.058998,0.070373,0.012006,104.907982,59.005108,0.086990,-0.024002,"[1863, 1864, 1771, 1772]","[6739, 6831, 6740, 6832]"
...,...,...,...,...,...,...,...,...,...,...,...,...
15809,30,638.00,147.348999,44.167999,0.003645,0.001537,5.121597,1.838781,0.029999,-0.000999,"[5243, 5335, 5151, 5427]","[3495, 3587, 3403, 3679]"
15810,30,638.25,147.307007,44.138000,0.003562,0.001569,2.039352,2.796997,-0.041992,-0.029999,"[5243, 5335, 5151, 5427]","[3495, 3587, 3403, 3679]"
15811,30,638.50,147.250000,44.146999,0.003808,0.000913,2.834540,9.775051,-0.057007,0.008999,"[5243, 5335, 5151, 5427]","[3495, 3587, 3403, 3679]"
15812,30,638.75,147.322998,44.201000,0.003603,0.000887,1.315185,12.162552,0.072998,0.054001,"[5243, 5335, 5151, 5427]","[3495, 3403, 3587, 3311]"


In [15]:
main.to_csv("data/total_add_velocity_linear_interpolation_vector_field_polar_coordinate.csv", index = False)

In [16]:
main.describe()

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati
count,15814.000000,15814.000000,15814.000000,15814.000000,14038.000000,14038.000000,15814.000000,15814.000000,15814.000000,15814.000000
mean,15.866384,341.630644,134.255299,38.418786,0.016949,0.017317,17.512056,17.682753,0.014679,0.006684
std,8.613671,140.691731,3.622366,2.460060,0.022497,0.021103,20.079624,20.079016,0.053495,0.043642
min,0.000000,51.500000,125.299004,33.140999,0.000020,0.000029,0.008665,0.009184,-0.230011,-0.247002
25%,9.000000,240.250000,131.248001,36.632000,0.004409,0.004294,3.683711,3.905549,-0.016998,-0.016998
50%,16.000000,348.000000,133.824501,37.919998,0.009696,0.010012,10.449545,11.003186,0.010010,0.006001
75%,23.000000,436.250000,136.547752,39.633999,0.021098,0.021817,23.913696,24.190725,0.043991,0.031002
max,30.000000,666.000000,148.697998,46.507000,0.701731,0.267044,169.434449,207.278049,0.475006,0.247002
